In [ ]:
import numpy as np
import sympy as sp

# 1. ОПРЕДЕЛЕНИЕ ФУНКЦИЙ (БЕЗ ЗАВИСИМОСТИ ОТ N_user)

# Весовая функция
def rho(x):
    return np.sqrt(x)

# Функции для тестирования
def f0(x):  # многочлен степени 0
    return 1.0

def f1(x):  # многочлен степени 1
    return 2*x + 1

def f2(x):  # многочлен степени 2
    return 3*x**2 - 2*x + 1

def f3(x):  # многочлен степени 3
    return 4*x**3 - 3*x**2 + 2*x - 1

def f4(x, N):  # произвольный многочлен степени N-1
    # Например: x^(N-1) + x^(N-2) + ... + x + 1
    result = 0
    for i in range(N):
        result += x**i
    return result

def f5(x):  # функция из варианта
    return np.sin(x)

def main():

    # 2. ВЗАИМОДЕЙСТВИЕ С ПОЛЬЗОВАТЕЛЕМ

    
    print("=" * 50)
    print("ЗАДАНИЕ 4.1: ПОСТРОЕНИЕ ИНТЕРПОЛЯЦИОННОЙ КФ")
    print("Весовая функция: ρ(x) = sqrt(x)")
    print("=" * 50)
    
    # Ввод параметров
    a = float(input("Введите левый конец промежутка интегрирования a: ") or 0)
    b = float(input("Введите правый конец промежутка интегрирования b: ") or 1)
    
    print("\nДоступные функции для интегрирования:")
    print("0: f0(x) = 1 (многочлен степени 0)")
    print("1: f1(x) = 2x + 1 (многочлен степени 1)")
    print("2: f2(x) = 3x^2 - 2x + 1 (многочлен степени 2)")
    print("3: f3(x) = 4x^3 - 3x^2 + 2x - 1 (многочлен степени 3)")
    print("4: f4(x) = x^(N-1) + ... + x + 1 (многочлен степени N-1)")
    print("5: f5(x) = sin(x) (функция из варианта)")
    
    func_choice = int(input("\nВыберите функцию для интегрирования (0-5): ") or 5)
    N_user = int(input("Введите количество узлов N: ") or 5)
    
    # Ввод узлов
    print(f"\nВведите {N_user} узлов (разделенных пробелами):")
    default_nodes = " ".join([str(0.1 + 0.2*i) for i in range(N_user)]) if N_user <= 5 else "0.1 0.3 0.5 0.7 0.9"
    nodes_input = input(f"Узлы x1 x2 ... x{N_user} [по умолчанию: {default_nodes}]: ") or default_nodes
    x_nodes = list(map(float, nodes_input.split()))
    
    if len(x_nodes) != N_user:
        print(f"Ошибка: введено {len(x_nodes)} узлов вместо {N_user}. Использую первые {N_user}.")
        x_nodes = x_nodes[:N_user]
    
    print(f"\nВыбранные узлы: {x_nodes}")

    # 3. ВЫЧИСЛЕНИЕ МОМЕНТОВ

    def compute_moments(a, b, N):
        """Вычисление моментов mu_k = ∫_a^b x^k * ρ(x) dx для k=0..N-1"""
        x_sym = sp.symbols('x')
        mu = []
        for k in range(N):
            integrand = x_sym**k * sp.sqrt(x_sym)  # x^k * sqrt(x)
            mu_k = sp.integrate(integrand, (x_sym, a, b))
            mu.append(float(mu_k))
        return mu
    
    print("\n" + "="*50)
    print("ВЫЧИСЛЕНИЕ МОМЕНТОВ")
    print("="*50)
    
    mu = compute_moments(a, b, N_user)
    print(f"Моменты μ_0 ... μ_{N_user-1}:")
    for i, m in enumerate(mu):
        print(f"  μ_{i} = {m:.6f}")

    # 4. ПОСТРОЕНИЕ СИСТЕМЫ УРАВНЕНИЙ

    
    print("\n" + "="*50)
    print("РЕШЕНИЕ СЛАУ ДЛЯ КОЭФФИЦИЕНТОВ")
    print("="*50)
    
    # Матрица Вандермонда
    V = np.vander(x_nodes, N_user, increasing=True).T
    
    print("Матрица Вандермонда:")
    print(np.array_str(V, precision=4, suppress_small=True))
    
    print(f"\nВектор правых частей (моменты): {np.array(mu)}")
    
    # Решение системы
    A = np.linalg.solve(V, mu)
    
    print(f"\nКоэффициенты ИКФ:")
    for i, coeff in enumerate(A):
        print(f"  A_{i+1} = {coeff:.8f}")
    

    # 5. ПРОВЕРКА ТОЧНОСТИ НА МНОГОЧЛЕНЕ СТЕПЕНИ N-1

    
    print("\n" + "="*50)
    print("ПРОВЕРКА ТОЧНОСТИ НА МНОГОЧЛЕНЕ СТЕПЕНИ N-1")
    print("="*50)
    
    # Создаем многочлен степени N-1 со случайными коэффициентами
    np.random.seed(42)
    coeffs_poly = np.random.randn(N_user)
    
    def poly_func(x):
        result = 0
        for i in range(N_user):
            result += coeffs_poly[i] * x**i
        return result
    
    # "Точное" значение интеграла от многочлена
    x_sym = sp.symbols('x')
    integrand_exact = 0
    for i in range(N_user):
        integrand_exact += coeffs_poly[i] * x_sym**i * sp.sqrt(x_sym)
    
    I_exact_poly = float(sp.integrate(integrand_exact, (x_sym, a, b)))
    
    # Приближенное значение по ИКФ
    I_approx_poly = 0
    for i in range(N_user):
        I_approx_poly += A[i] * poly_func(x_nodes[i])
    
    print(f"Многочлен степени {N_user-1}: P(x) = ", end="")
    for i in range(N_user):
        if i == 0:
            print(f"{coeffs_poly[i]:.3f} ", end="")
        else:
            print(f"+ {coeffs_poly[i]:.3f}*x^{i} ", end="")
    print()
    
    print(f"Точное значение ∫ρ(x)P(x)dx = {I_exact_poly:.10f}")
    print(f"Приближенное значение = {I_approx_poly:.10f}")
    print(f"Разность = {abs(I_exact_poly - I_approx_poly):.2e}")
    
    if abs(I_exact_poly - I_approx_poly) < 1e-10:
        print(" ИКФ точна для многочлена степени N-1 (как и должно быть)")
    else:
        print(" Внимание: небольшая погрешность из-за округлений")

    # 6. ВЫЧИСЛЕНИЕ ИНТЕГРАЛА ДЛЯ ВЫБРАННОЙ ФУНКЦИИ

    
    print("\n" + "="*50)
    print("ВЫЧИСЛЕНИЕ ИНТЕГРАЛА ДЛЯ ВЫБРАННОЙ ФУНКЦИИ")
    print("="*50)
    
    # Выбор функции
    if func_choice == 0:
        func = f0
        func_desc = "f0(x) = 1 (многочлен степени 0)"
    elif func_choice == 1:
        func = f1
        func_desc = "f1(x) = 2x + 1 (многочлен степени 1)"
    elif func_choice == 2:
        func = f2
        func_desc = "f2(x) = 3x^2 - 2x + 1 (многочлен степени 2)"
    elif func_choice == 3:
        func = f3
        func_desc = "f3(x) = 4x^3 - 3x^2 + 2x - 1 (многочлен степени 3)"
    elif func_choice == 4:
        func = lambda x: f4(x, N_user)
        func_desc = f"f4(x) = x^{N_user-1} + ... + x + 1 (многочлен степени {N_user-1})"
    else:  # func_choice == 5
        func = f5
        func_desc = "f5(x) = sin(x) (функция из варианта)"
    
    print(f"Выбрана функция: {func_desc}")
    
    # "Точное" значение интеграла
    x_sym = sp.symbols('x')
    
    if func_choice == 5:
        # sin(x) * sqrt(x)
        integrand_exact = sp.sin(x_sym) * sp.sqrt(x_sym)
        I_exact = float(sp.integrate(integrand_exact, (x_sym, a, b)))
    elif func_choice == 4:
        # Многочлен степени N-1
        integrand_exact = 0
        for i in range(N_user):
            integrand_exact += x_sym**i * sp.sqrt(x_sym)
        I_exact = float(sp.integrate(integrand_exact, (x_sym, a, b)))
    else:
        # Для явных многочленов f0-f3
        # f0: ∫√x dx = 2/3 * x^(3/2)
        # f1: ∫(2x+1)√x dx = ∫(2x^(3/2) + √x) dx = 4/5 x^(5/2) + 2/3 x^(3/2)
        # и т.д.
        # Для простоты используем численное интегрирование
        try:
            if func_choice == 0:
                integrand_exact = sp.sqrt(x_sym)
            elif func_choice == 1:
                integrand_exact = (2*x_sym + 1) * sp.sqrt(x_sym)
            elif func_choice == 2:
                integrand_exact = (3*x_sym**2 - 2*x_sym + 1) * sp.sqrt(x_sym)
            elif func_choice == 3:
                integrand_exact = (4*x_sym**3 - 3*x_sym**2 + 2*x_sym - 1) * sp.sqrt(x_sym)
            I_exact = float(sp.integrate(integrand_exact, (x_sym, a, b)))
        except:
            # Численное интегрирование с высокой точностью
            from scipy import integrate
            def integrand_num(x):
                return rho(x) * func(x)
            I_exact, _ = integrate.quad(integrand_num, a, b, limit=100, epsabs=1e-14)
    
    # Приближенное значение по ИКФ
    I_approx = 0
    for i in range(N_user):
        I_approx += A[i] * func(x_nodes[i])
    
    print(f"Точное значение ∫ρ(x)f(x)dx = {I_exact:.12f}")
    print(f"Приближенное значение = {I_approx:.12f}")
    
    # Погрешности
    abs_error = abs(I_exact - I_approx)
    rel_error = abs_error / abs(I_exact) if I_exact != 0 else abs_error
    
    print(f"\nАбсолютная погрешность = {abs_error:.2e}")
    print(f"Относительная погрешность = {rel_error:.2e}")

    # 7. ВЫВОД РЕЗУЛЬТАТОВ

    
    print("\n" + "="*50)
    print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
    print("="*50)
    
    print(f"Параметры:")
    print(f"  Промежуток: [{a}, {b}]")
    print(f"  Вес: ρ(x) = sqrt(x)")
    print(f"  Узлы: {x_nodes}")
    print(f"  Коэффициенты: {A}")
    
    print(f"\nРезультат для {func_desc}:")
    print(f"  Точное значение: {I_exact:.10f}")
    print(f"  Приближенное:    {I_approx:.10f}")
    print(f"  Абс. погрешность: {abs_error:.2e}")
    

    # 8. ВОЗМОЖНОСТЬ ПОВТОРИТЬ

    
    print("\n" + "="*50)
    choice = input("Хотите выбрать новую функцию/параметры? (y/n): ")
    if choice.lower() == 'y':
        print("\n" + "="*50)
        main()  # Рекурсивный вызов для нового расчета
    else:
        print("Программа завершена.")

# Запуск программы
if __name__ == "__main__":
    main()

ЗАДАНИЕ 4.1: ПОСТРОЕНИЕ ИНТЕРПОЛЯЦИОННОЙ КФ
Весовая функция: ρ(x) = sqrt(x)


Введите левый конец промежутка интегрирования a:  0
Введите правый конец промежутка интегрирования b:  1



Доступные функции для интегрирования:
0: f0(x) = 1 (многочлен степени 0)
1: f1(x) = 2x + 1 (многочлен степени 1)
2: f2(x) = 3x^2 - 2x + 1 (многочлен степени 2)
3: f3(x) = 4x^3 - 3x^2 + 2x - 1 (многочлен степени 3)
4: f4(x) = x^(N-1) + ... + x + 1 (многочлен степени N-1)
5: f5(x) = sin(x) (функция из варианта)



Выберите функцию для интегрирования (0-5):  5
Введите количество узлов N:  6



Введите 6 узлов (разделенных пробелами):


Узлы x1 x2 ... x6 [по умолчанию: 0.1 0.3 0.5 0.7 0.9]:  0 0.2 0.4 0.6 0.8 1



Выбранные узлы: [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

ВЫЧИСЛЕНИЕ МОМЕНТОВ
Моменты μ_0 ... μ_5:
  μ_0 = 0.666667
  μ_1 = 0.400000
  μ_2 = 0.285714
  μ_3 = 0.222222
  μ_4 = 0.181818
  μ_5 = 0.153846

РЕШЕНИЕ СЛАУ ДЛЯ КОЭФФИЦИЕНТОВ
Матрица Вандермонда:
[[1.     1.     1.     1.     1.     1.    ]
 [0.     0.2    0.4    0.6    0.8    1.    ]
 [0.     0.04   0.16   0.36   0.64   1.    ]
 [0.     0.008  0.064  0.216  0.512  1.    ]
 [0.     0.0016 0.0256 0.1296 0.4096 1.    ]
 [0.     0.0003 0.0102 0.0778 0.3277 1.    ]]

Вектор правых частей (моменты): [0.66666667 0.4        0.28571429 0.22222222 0.18181818 0.15384615]

Коэффициенты ИКФ:
  A_1 = 0.01506642
  A_2 = 0.08676509
  A_3 = 0.15836016
  A_4 = 0.09176009
  A_5 = 0.25234025
  A_6 = 0.06237466

ПРОВЕРКА ТОЧНОСТИ НА МНОГОЧЛЕНЕ СТЕПЕНИ N-1
Многочлен степени 5: P(x) = 0.497 + -0.138*x^1 + 0.648*x^2 + 1.523*x^3 + -0.234*x^4 + -0.234*x^5 
Точное значение ∫ρ(x)P(x)dx = 0.7207475843
Приближенное значение = 0.7207475843
Разность = 1.11e-16
✓ ИКФ т